In [ ]:
import os
op = os.path
opd = op.dirname
os.chdir(opd(os.getcwd()))

In [ ]:
%matplotlib
##
import yaml
from pathlib import Path
from time import sleep, time
import numpy as np
from matplotlib import pyplot as plt
import cv2
##
from Olympuslx81 import OLYMP
from Prior import PRIOR
from Evolve512 import EVOLVE
###
from tensorflow.keras import models
import tensorflow.keras as tfk

In [ ]:
ol = OLYMP()
pr = PRIOR()
ev = EVOLVE()

In [ ]:
dic_mod = {'ep5_v3' : 'training-cell_v3-ep5-bs4-fl_date04-10-2020-13-18'}
mod = models.load_model( Path('..') / 'models' / dic_mod['ep5_v3'] ) 

In [ ]:
def take_pic_posz(i, val, move_type=None, show=False):
    '''
    take the picture at pos posz
    '''
    ol.go_zpos( val, move_type=move_type )            # 797300 
    answ = ol.receive(13)
    print(f'answ 0 {answ}')
    #addr_img = f'uu{i}.tiff'
    addr_img = f'bf_for_optimz{i}.tiff'
    ev.take_pic(addr_img, bpp=8, exp_time=100)
    ev.adapt()
    if show: plt.imshow(ev.frame, cmap='gray')
    return ev.frame
        
def make_pred(i, thresh=253, show=False):
    '''
    make the prediction and return surface 
    '''
    img = cv2.imread(f'bf_for_optimz{i}.tiff')
    arr = np.array([ img ], dtype=np.float32)/255                        # format array for prediction
    pred = mod.predict(arr)[0]*255
    ##
    ll = len( np.where( pred > thresh )[0] )
    print(f'll { ll }')
    addr_pred = f'pred_uu{i}.png'
    cv2.imwrite(addr_pred, pred)
    if show:
        plt.figure()
        plt.imshow(pred, cmap='gray')
    return ll

In [ ]:
def find_z_optim():
    '''
    find the optimal focus
    '''
    t0 = time()
    l_surf = []
    ref = 785400
    step = 200
    ol.go_zpos(ref)
    for i in range(9):
        print(f'i is { i } ')
        take_pic_posz(i, ref + i*step, 'd')
        l_surf += [ make_pred(i) ]

    plt.plot(l_surf)
    plt.show()
    t1 = time()
    print(f'time elapsed is { round((t1-t0),1) } s')
    plt.savefig('optim_focus.png')
    ind_max = l_surf.index(max(l_surf))
    print(f'ind_max is { ind_max }')

In [ ]:
find_z_optim()